# The mission

We want to apply the homologous outflow equations to a HD frame that is in the homologous outflow stage such that it can expand using said equations.

# 0. Setting everything up

We first import necessary libraries.

In [1]:
import yt
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as unit
import astropy.constants as const
%matplotlib notebook

We now define all necessary functions (homologous outflow).

In [2]:
def omega(v):
    '''
    Return $\hat{\Omega} = \dot{r}/|\dot{r}|$ for a cell grid. 
    This quantity is the unit first time derivative of the radius vector.
    
    INPUT: 
    
    --v (numpy array): (n, n, 2) shaped array containing x,y components of 
    velocity vector for an n by n grid.
    
    OUTPUT: 
    
    --omega (numpy array): (n, n, 2) vector containing x,y components of 
    velocity unit vector for an n by n grid.
    '''
    return v / np.linalg.norm(v, axis = 1)[:,None]

def s(t_sim, t):
    '''
    Return $s = c (t-t_{sim})$ in 2D. 
    This quantity is the step size the cell takes when moving at 
    speed $c$ from time $t_{sim}$ to time $t$. 
    
    INPUT: 
    
    --t_sim (float): initial time
    
    --t (float): final time
    
    OUTPUT: 
    
    --s float: magnitude of step taken in time $(t-t_{sim}})$ given by velocity $c$.
    '''

    return ( ( t - t_sim ) * const.c.cgs.value )

def r_step_c(r_i,s,omega_hat):
    '''
    Return $r = r_i + s \hat{\Omega}$ in 2D. 
    This quantity is the step size the cell takes when moving a distance
    given by s from time $t_{sim}$ to time $t$
    
    INPUT: 
    
    --r_i (numpy array)
    
    --t_sim (float): initial time
    
    --t (float): final time
    
    OUTPUT: 
    
    --s float: magnitude of step taken in time $(t-t_{sim}})$
    '''
    return ( r_i + s*omega_hat )

def dedr(e,gamma,n,r):
    return  -(8*(e**(3/4))*(4*e + 3*n))/(r*((9*n/(e**(1/4)) + 8*(e**(3/4))))) 

def dgammadr(e,gamma,n,r):
    return (8*(e**(3/4))*gamma)/(r*((9*n/(e**(1/4)) + 8*(e**(3/4)))))

def dndr(e,gamma,n,r):
    return -(6*n*(4*e + 3*n))/(e**(1/4)*r*((9*n/(e**(1/4)) + 8*(e**(3/4)))))

def homologous_outflow( gam, dens, r0, rf, ener = None, h_i = []):
    '''
    Return the solution to ODE soln. vectors (r, energy, gamma, density) 
    from r0 to rf.
    Integrates set of homologous outflow equations with initial conditions 
    
    ener (erg cm^-3 c^-2 = g cm^-3 energy density over c^2), 
    
    gam (unitless Lorentz factor), 
    
    n (g cm^-3 mass density),
    
    from r0 to rf (cm).
    
    INPUT: 
    
    --gam (numpy array): matrix containing a grid for initial Lorentz Factor
    
    --dens (numpy array): matrix containing a grid for initial mass density
    
    --r0 (numpy array): matrix containing a grid for initial radius
    
    --rf (numpy array): matrix containing a grid for final radius
    
    --ener (numpy array): matrix containing a grid for initial energy/$c^2$, default = None
    
    --h_i (numpy array): matrix containing a grid for initial specific enthalpy, default = None
    
    OUTPUT: 
    
    --r_step (numpy array): matrix containing a grid with vectors 
    containing radius discretization for solution vectors
    
    --e_arr (numpy array): matrix containing a grid with vectors 
    containing solution for energy/$c^2$ as a function of radius
    
    --gamma_arr (numpy array): matrix containing a grid with vectors 
    containing solution for Lorentz factor as a function of radius
    
    --n_arr (numpy array): matrix containing a grid with vectors 
    containing solution for mass density as a function of radius
    
    --h_arr (numpy array): matrix containing a grid with vectors 
    containing solution for specific enthalpy as a function of radius
    
    NOTES:
    
    --Even though the parameter defaults for ener and h_i are None, the function needs one or the other
    in order to retrieve the solution vectors. if ener is given, it will take it as ICs. If h_i is given,
    the function calculates energy from this and proceeds to solve the ODEs with this calculated IC.
    '''
    
    if ener == None: 
        
        ener = 3 * ( h_i - 1 ) * dens / 4 
        
        h_start = h_i
        
    if len(h_i) == 0:
        
        h_start = (ener * 4 / (dens * 3) ) + 1
    
    e_start = ener #e_tilde #e/const.c.cgs.value**2
    gamma_start = gam
    n_start = dens #/const.m_p.cgs.value

    
    
    steps = 10000

    diff = (rf - r0)/(steps-1)

    r_step = np.zeros((steps, r0.shape[0]))

    r_step[0] = r0

    for i in range(1, steps):
    
        r_step[i] = r_step[i-1] + diff

    
    
    e_arr = np.zeros(r_step.shape)       # create an array of zeros for e
    gamma_arr = np.zeros(r_step.shape)   # create an array of zeros for gamma
    n_arr = np.zeros(r_step.shape)       # create an array of zeros for n
    h_arr = np.zeros(r_step.shape)
    e_arr[0] = e_start                  # add initial value of e to array
    gamma_arr[0] = gamma_start          # add initial value of gamma to array
    n_arr[0] = n_start                  # add initial value of n to array
    h_arr[0] = h_start

    for i in range (1, r_step.shape[0]):  
        e = e_arr[i-1]
        gamma = gamma_arr[i-1]
        n = n_arr[i-1]
        r = r_step[i-1]
        Dr = r_step[i] - r_step[i-1]

        #calculate k1 for e, gamma and n
        k1_e = dedr(e,gamma,n,r)
        k1_gamma = dgammadr(e,gamma,n,r)
        k1_n = dndr(e,gamma,n,r)

        #calculate k2 for e, gamma and n
        k2_e = dedr(e+(1/2)*Dr*k1_e,gamma+(1/2)*Dr*k1_gamma,n+(1/2)*Dr*k1_n,r+(1/2)*Dr)
        k2_gamma = dgammadr(e+(1/2)*Dr*k1_e,gamma+(1/2)*Dr*k1_gamma,n+(1/2)*Dr*k1_n,r+(1/2)*Dr)
        k2_n = dndr(e+(1/2)*Dr*k1_e,gamma+(1/2)*Dr*k1_gamma,n+(1/2)*Dr*k1_n,r+(1/2)*Dr)

        #calculate k3 for e, gamma and n
        k3_e = dedr(e+(1/2)*Dr*k2_e,gamma+(1/2)*Dr*k2_gamma,n+(1/2)*Dr*k2_n,r+(1/2)*Dr)
        k3_gamma = dgammadr(e+(1/2)*Dr*k2_e,gamma+(1/2)*Dr*k2_gamma,n+(1/2)*Dr*k2_n,r+(1/2)*Dr)
        k3_n = dndr(e+(1/2)*Dr*k2_e,gamma+(1/2)*Dr*k2_gamma,n+(1/2)*Dr*k2_n,r+(1/2)*Dr)

        #calculate k4 for e, gamma and n
        k4_e = dedr(e+Dr*k3_e,gamma+Dr*k3_gamma,n+Dr*k3_n,r+Dr)
        k4_gamma = dgammadr(e+Dr*k3_e,gamma+Dr*k3_gamma,n+Dr*k3_n,r+Dr)
        k4_n = dndr(e+Dr*k3_e,gamma+Dr*k3_gamma,n+Dr*k3_n,r+Dr)

        e_arr[i] = e + (1/6)*Dr*(k1_e+2*k2_e+2*k3_e+k4_e)                   # calc. e at next radius step,add to array
        gamma_arr[i] = gamma + (1/6)*Dr*(k1_gamma+2*k2_gamma+2*k3_gamma+k4_gamma) # calc. gamma at next radius step,add to array
        n_arr[i] = n + (1/6)*Dr*(k1_n+2*k2_n+2*k3_n+k4_n)           # calc. n at next radius tep,add to array
        
        h_arr[i] = 1+ (4*e_arr[i])/(3*n_arr[i])
    
    return (r_step, e_arr, gamma_arr, n_arr, h_arr)

# 1. We now import a HD frame

We use yt to read a FLASH frame from a 16TI model. We use the final frame in our simulation, since we predict the outflow has reached a homologous state at this point in the simulation.

In [3]:
frame_flash = yt.load(
  '/Users/josearita-escalante/Documents/GRB-NASA/2nd-project/notebooks/hd-frames/16TI/FLASH/rhd_jet_big_13_hdf5_plt_cnt_3092')

yt : [WARNING  ] 2023-06-07 13:05:09,125 Guessing dimensionality as 2
yt : [WARNING  ] 2023-06-07 13:05:09,126 Extending theta dimension to 2PI + left edge.
yt : [INFO     ] 2023-06-07 13:05:09,226 Parameters: current_time              = 18558.002
yt : [INFO     ] 2023-06-07 13:05:09,227 Parameters: domain_dimensions         = [160 800   1]
yt : [INFO     ] 2023-06-07 13:05:09,229 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-06-07 13:05:09,231 Parameters: domain_right_edge         = [5.12000000e+03 2.56000000e+04 6.28318531e+00]
yt : [INFO     ] 2023-06-07 13:05:09,231 Parameters: cosmological_simulation   = 0


In [4]:
frame_flash.field_list

[('flash', 'dens'),
 ('flash', 'ener'),
 ('flash', 'pres'),
 ('flash', 'velx'),
 ('flash', 'vely')]

In [5]:
frame_flash.derived_field_list

[('flash', 'cell_volume'),
 ('flash', 'dens'),
 ('flash', 'dr'),
 ('flash', 'dtheta'),
 ('flash', 'dz'),
 ('flash', 'ener'),
 ('flash', 'path_element_r'),
 ('flash', 'path_element_theta'),
 ('flash', 'path_element_z'),
 ('flash', 'pres'),
 ('flash', 'r'),
 ('flash', 'theta'),
 ('flash', 'velx'),
 ('flash', 'vely'),
 ('flash', 'volume'),
 ('flash', 'z'),
 ('gas', 'averaged_density'),
 ('gas', 'cell_mass'),
 ('gas', 'cell_volume'),
 ('gas', 'density'),
 ('gas', 'density_gradient_magnitude'),
 ('gas', 'density_gradient_r'),
 ('gas', 'density_gradient_theta'),
 ('gas', 'density_gradient_z'),
 ('gas', 'dr'),
 ('gas', 'dtheta'),
 ('gas', 'dynamical_time'),
 ('gas', 'dz'),
 ('gas', 'four_velocity_magnitude'),
 ('gas', 'four_velocity_r'),
 ('gas', 'four_velocity_t'),
 ('gas', 'four_velocity_z'),
 ('gas', 'lorentz_factor'),
 ('gas', 'mach_number'),
 ('gas', 'mass'),
 ('gas', 'momentum_density_r'),
 ('gas', 'momentum_density_z'),
 ('gas', 'momentum_r'),
 ('gas', 'momentum_z'),
 ('gas', 'path_ele

# 2. Let's analyze the data

We get quantities such as the ( r , z ) components, density, energy, velocity ($v_r$, $v_z$) components.

We first need to convert all quantities containing length from code length to centimeters, by multiplying them by $10^9$. 

Let's first do position (r and z). We can use these two components to get the real radius.

In [6]:
ad = frame_flash.all_data()

z = ad['flash','z']

r = ad['flash','r']

In [7]:
r_units = r*1e9

print(r.min(),r.max())

print(r_units.min(),r_units.max())

0.00390625 cm 5119.0 cm
3906250.0 cm 5119000000000.0 cm


In [8]:
z_units = z*1e9

print(z.min(),z.max())

print(z_units.min(),z_units.max())

0.00390625 cm 25584.0 cm
3906250.0 cm 25584000000000.0 cm


In [9]:
r_comps = np.array([r_units, z_units]).T

r_mag = np.linalg.norm(r_comps, axis = 1)

We now do velocity. These velocity components are multiples of $c$. We can get the Lorentz factor from this.

In [10]:
v_f_x = ad['flash', 'velx']

v_f_y = ad['flash', 'vely']

In [11]:
print(v_f_x)

print(v_f_y)

[3.38382015e-05 3.40338884e-05 3.40608312e-05 ... 2.24541754e-01
 2.24491894e-01 2.24455371e-01] code_length/code_time
[0.99993378 0.99994081 0.99994689 ... 0.89574683 0.89625514 0.89674795] code_length/code_time


In [12]:
v_comps = np.array([v_f_x, v_f_y]).T

v_mag = np.linalg.norm(v_comps, axis = 1)

In [13]:
lor_fac = 1 / np.sqrt(1-v_mag**2)

lor_fac

array([86.89544028, 91.91340963, 97.03183313, ...,  2.60626532,
        2.61416583,  2.62194677])

Now we load energy. The flash energy (even though yt says otherwise), is already in units of erg c$^{-2}$ cm$^{-3}$.

In [14]:
ener = ad['flash','ener']

ener

unyt_array([7.57024219e+03, 8.46783203e+03, 9.43574121e+03, ...,
            7.07993889e+00, 7.12484932e+00, 7.16882181e+00], 'code_length**2/code_time**2')

Pressure is in the same units as energy and defined as $e = 3 p$. This is not what we see, not in the units or in the values themselves. 

In [15]:
pres = ad['flash','pres']

pres

unyt_array([2.08381244e-15, 2.06894574e-15, 2.15462079e-15, ...,
            1.91823705e-12, 1.93090422e-12, 1.94020581e-12], 'code_mass/(code_length*code_time**2)')

We now look at density. This has mass over volume units. 

In [16]:
dens = ad['flash','dens']

dens

unyt_array([3.85595115e-12, 4.06114725e-12, 4.27694382e-12, ...,
            1.74721834e-10, 1.74755238e-10, 1.74744136e-10], 'code_mass/code_length**3')

Let's test the h values. These do not seem quite right for energy.

In [17]:
h_ener = 1 + np.array(ener) * 4 / (3* np.array(dens))

print(h_ener.min(), h_ener.max())

7.864207906365611 2.274306036194676e+19


In [18]:
h_pres = 1 + np.array(pres) * 4 / ( np.array(dens))

print(h_pres.min(), h_pres.max())

1.0000000000281144 114.6647588100743


It seems like pressure gives us right values for enthalpy. Let's try the solver with these values.

In [ ]:
new_r_flash = r_step_c(r_comps, s(10,100 + 10), omega(v_comps))

rf_flash = np.linalg.norm(new_r_flash, axis = 1)

r_flash, e_flash, g_flash, n_flash, h_flash = homologous_outflow(
  lor_fac, 
  dens, 
  r_mag, 
  rf_flash, 
  h_i = h_pres)

In [ ]:
fig = plt.figure()

ax = plt.gca()

mapp = ax.scatter(r_comps[:,0],r_comps[:,1], c = lor_fac)

cb = plt.colorbar(mappable = mapp, ax = ax)

plt.axis('equal')

In [ ]:
fig = plt.figure()

ax = plt.gca()

mapp1 = ax.scatter(new_r_flash[:,0],new_r_flash[:,1], c = g_flash[-1])

cb1 = plt.colorbar(mappable = mapp1, ax = ax)

plt.axis('equal')